## Entrainement d'un modèle avec la méthode des SVM

#### Chargement des données d'entrainement

In [1]:
import os
import cv2
import numpy as np
import random

In [2]:
""" 
We will create a dict with all the usefull datas of the training dataset
datas = {
    "XXXX" (name of the file) : {
        "img" : ndarray of the image,
        "labels" (data of the labels): {
            "X" index of the label (0,1,...,n) : {
                "name" : name of the label,
                "coord" : coord of the label like xmin, ymin, xmax, ymax,
                "img" : crooped img of the label,
            }
        }
    }
}

"""

def generate_empty_bbox(image_width, image_height):
    # Thanks to the stats, we know that size of bbox will be (127, 145) -> Average size of labels 
    # Génération de coordonnées aléatoires pour le coin supérieur gauche de la boundebox
    x_min = random.randint(0, image_width - 127)
    y_min = random.randint(0, image_height - 145)
    
    # Calcul des coordonnées du coin inférieur droit de la boundebox
    x_max = x_min + 127
    y_max = y_min + 145
    
    return (x_min, y_min, x_max, y_max)

def load_data(image_dir, label_dir):
    datas = {}

    for image_file in os.listdir(image_dir):
        # Computing name and files paths
        image_path = image_dir + '/' + image_file
        name = image_file.split('.')[0]
        label_path = label_dir + '/' + name + '.csv'
        
        # Import image as array
        image = cv2.imread(image_path)

        # Import labels as array 
        with open(label_path, 'r') as file:
            rows = file.readlines()

            label_data = {}
            if rows == ['\n']:  # Create a random empty label to balance model
                # Create random coords for empty label
                xmin, ymin, xmax, ymax = generate_empty_bbox(image.shape[1], image.shape[0])
    
                # Get the cropped image (as array) of the label
                cropped_image = image[ymin:ymax, xmin:xmax]
               
                label_data[0] = {
                        "name":"empty",
                        "coord": (xmin, ymin, xmax, ymax),
                        "img":cropped_image
                    }
            else:
                for i, row in enumerate(rows):  # One image can contain several labels
                    row = row.strip().split(",")

                    # Compute coords of the label
                    xmin, ymin, xmax, ymax = map(int, row[0:4])

                    # Get the label name
                    class_name = row[4]

                    # Get the cropped image (as array) of the label
                    cropped_image = image[ymin:ymax, xmin:xmax]
                    
                    # Adding to the json
                    label_data[i] = {
                        "name":class_name,
                        "coord": (xmin, ymin, xmax, ymax),
                        "img":cropped_image
                    }

        datas[name] = {
             "img" : image,
             "labels" : label_data,
        }
       
    return datas

In [3]:
# Creating the dict of the datas 

datas = load_data("../data/train/images", "../data/train/labels")

In [4]:
def extract_features(img):
    # Convertion to gray level
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Color Hist
    hist_color = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist_color = cv2.normalize(hist_color, hist_color).flatten()
    
    # Gradient Hist
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    grad_mag = np.sqrt(sobelx**2 + sobely**2)
    hist_gradient = cv2.calcHist([grad_mag.astype(np.uint8)], [0], None, [16], [0, 256])
    hist_gradient = cv2.normalize(hist_gradient, hist_gradient).flatten()
    
    return np.concatenate((hist_color, hist_gradient))


# Dict to convert str class name to int
name_to_int = {
    "danger": 0,
    "interdiction": 1,
    "obligation": 2,
    "stop": 3,
    "ceder": 4,
    "frouge": 5,
    "forange": 6,
    "fvert": 7,
    "ff": 8,
    "empty": 9
}


# Creating arrays with all labels datas & classes
X_train = []
Y_train = []

for name, data in datas.items():
    for row in data["labels"].values():
        X_train.append(extract_features(row["img"]))
        Y_train.append(name_to_int[row["name"]])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [6]:
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

svm_model = svm.SVC(kernel='linear')  # Choix du noyau linéaire
svm_model.fit(X_train, Y_train)

print(svm_model)

[1 1 0 ... 1 5 7]
